In [5]:
from selenium                               import webdriver
from webdriver_manager.chrome               import ChromeDriverManager
from selenium.webdriver.chrome.service      import Service
from selenium.webdriver.common.by           import By
from selenium.webdriver.chrome.options      import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *

from datetime import date
import time

import pandas as pd
import PyPDF2
import os

import warnings
warnings.filterwarnings('ignore')
#options.add_argument("--headless")  # Executa o Chrome em modo headless (sem GUI)

In [6]:
def criar_pasta_dados():
    # Identificar o diretório atual
    diretorio_atual = os.getcwd()
    
    # Caminho para a nova pasta
    pasta_dados = os.path.join(diretorio_atual, "dados")
    
    # Criar a nova pasta se ela não existir
    if not os.path.exists(pasta_dados):
        os.makedirs(pasta_dados)
        #print("Pasta 'dados' criada com sucesso.")
    #else:
        #print("A pasta 'dados' já existe.")
    
    # Retornar o caminho da pasta 'dados'
    return pasta_dados



# configurar selenium
def driver_settings(pasta_dados):
    # Configura web driver
    prefs = {'download.default_directory':f'{pasta_dados}', 
             'profile.default_content_setting_values.automatic_downloads': 1}

    # Set options
    options = Options()
    #options.add_argument("--headless")  # Executa o Chrome em modo headless (sem GUI)
    options.add_argument('start-maximized')
    options.add_argument('--disable-notifications')
    options.add_argument('--no-sandbox')
    options.add_argument('--verbose')
    options.add_experimental_option('prefs', prefs)


    # Run browser
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    return driver



def verificar_login_efetuado(driver):
    # redirecionar para home
    driver.get('https://drivers.uber.com/p3/payments/statements')

    # aguardar usuario efetuar login
    verificacao = False
    wait = WebDriverWait(driver, 10)

    while verificacao == False:
        try:
            # atribuit variavel ao wait?
            wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div[1]/div/div[2]/div/h5')))
            verificacao = True
            #print('login efetuado, iniciando download de PDFs')
        except:
            verificacao = False
            #print('login ainda nao efetudado')



def download_files(driver):
    wait = WebDriverWait(driver, 30)

    # Download files
    month = date.today().month
    year = date.today().year
    driver.get(f'https://drivers.uber.com/earnings/statements?month={month}&year={year}')
    
    # agraudar aparecer elemento com a classe
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_css-drSZEJ')))

    # contar links com a classe
    n_links = int(len(driver.find_elements(By.CLASS_NAME, '_css-drSZEJ')) / 2)

    # caso o mes atual ainda nao tenha relatorios vai passar para o anterior
    if n_links == 0:
        if month == 1:
            month = 12
            year = year - 1
        else:
            month = month - 1
        driver.get(f'https://drivers.uber.com/earnings/statements?month={month}&year={year}')

    x = True

    while x == True:
        try:           
            # agraudar aparecer elemento com a classe
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_css-drSZEJ')))

            n_links = int(len(driver.find_elements(By.CLASS_NAME, '_css-drSZEJ')) / 2)
            print(n_links)
            if n_links > 0:
                for link in range(1, n_links+1):             
                    print(f'download PDF | year: {year} | month: {month} | week: {link}/{n_links}')

                    # fazer download                               
                    download_pdf = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="wrapper"]/div[2]/div/div[2]/div[1]/div/table/tbody/tr[{link}]/td[4]/button')))
                    download_pdf.click()
           
                if month == 1:
                    month = 12
                    year = year - 1
                else:
                    month = month - 1
                
                # esperar que o ultimo item  esteja clicavel para caminhar para a proxima pagina
                time.sleep(10)
                driver.get(f'https://drivers.uber.com/earnings/statements?month={month}&year={year}')

            else:
                x = False
        except:
            print('no more pages to scrapping')
            x = False



def extract_links(pdf_path):
    # Extract links from pdf files              
    list_links = []
    for root, subFolder, filename in os.walk(pdf_path):
        for file in filename:
            PDFFile = open(f'{pdf_path}/{file}','rb')
            pdf = PyPDF2.PdfReader(PDFFile)
            pages = len(pdf.pages)
            key = '/Annots'
            uri = '/URI'
            ank = '/A'

            for page in range(pages):
                print("Current Page: {}".format(page+1))
                pageSliced = pdf.pages[page]
                pageObject = pageSliced.get_object()
                if key in pageObject.keys():
                    ann = pageObject[key]
                    for a in ann:
                        u = a.get_object()
                        if uri in u[ank].keys():
                            list_links.append(u[ank][uri])  
                        
    return list_links


In [7]:
# criar pasta de downoads
pasta_dados = criar_pasta_dados()

# Criar driver
driver = driver_settings(pasta_dados)

# aguardar usuario efetuar login
verificar_login_efetuado(driver)

#download_files(driver)

list_links = extract_links(pasta_dados)
len(list_links)

Current Page: 1
Current Page: 1
Current Page: 1
Current Page: 1
Current Page: 1
Current Page: 2
Current Page: 3
Current Page: 4
Current Page: 5
Current Page: 6
Current Page: 7
Current Page: 8
Current Page: 9
Current Page: 10
Current Page: 11
Current Page: 12
Current Page: 13
Current Page: 14
Current Page: 1
Current Page: 2
Current Page: 3
Current Page: 4
Current Page: 5
Current Page: 6
Current Page: 7
Current Page: 8
Current Page: 9
Current Page: 10
Current Page: 11
Current Page: 12
Current Page: 13
Current Page: 14
Current Page: 1
Current Page: 2
Current Page: 3
Current Page: 4
Current Page: 5
Current Page: 6
Current Page: 7
Current Page: 8
Current Page: 9
Current Page: 10
Current Page: 11
Current Page: 12
Current Page: 1
Current Page: 2
Current Page: 3
Current Page: 4
Current Page: 5
Current Page: 6
Current Page: 7
Current Page: 8
Current Page: 9
Current Page: 10
Current Page: 11
Current Page: 12
Current Page: 1
Current Page: 2
Current Page: 3
Current Page: 4
Current Page: 5
Current 

18461

In [8]:
# Scraping trips
df = pd.DataFrame(columns = [
        'DataHora',
        'Preco',
        'Duracao',
        'Distancia',
        'Origem',
        'Destino',
        'url'
        ])

print('Data frame was created')

# Error
list_objects_error = []

for link in list_links:
        try:
                # com preço definico
                driver.get(link)
                print(f'Start scrapping trip: {list_links.index(link) + 1} / {len(list_links)}')
                print('Objetos com erro:', len(list_objects_error))
                time.sleep(2)

                # Trip information                             //*[@id="wrapper"]/div/ol/li[1]/div[1]/p
                DataHora = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[1]/p[1]').text
                Preco = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[1]/h2').text
                Duracao = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[3]/div[1]/div[2]').text
                Distancia = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[3]/div[2]/div[2]').text
                Origem = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[4]/div[1]/div[2]/div/p').text
                Destino = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div/ol/li[1]/div[4]/div[2]/div[2]/div/p').text

                data = pd.DataFrame({
                        'DataHora':DataHora,
                        'Preco':Preco,
                        'Duracao':Duracao,
                        'Distancia':Distancia,
                        'Origem':Origem,
                        'Destino':Destino,
                        'url':link
                        }, index=[0])
                print('object created')
                        
                # Concat data to df
                df = pd.concat([df, data])
                print('Object added in data frame')
                print('')
        except:
               list_objects_error.append(link) # Simplificar o projeto para obtenção de resultado o quanto antes

# salvar dados extraidos das corridas
df.to_csv('dados/corridas_extraidas.csv', sep=';', encoding='utf-8', index=False)

Data frame was created
Start scrapping trip: 1 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 2 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 3 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 4 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 5 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 6 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 7 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 8 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 9 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 10 / 18461
Objetos com erro: 0
object created
Object added in data frame

Start scrapping trip: 11 /

In [9]:
df.tail()

,DataHora,Preco,Duracao,Distancia,Origem,Destino,url
0,UberX • 3 de out. de 2023 • 21:27,"R$ 31,09",35 min 22 segundos,18.3 km,"Rua das Seringueiras, Jabaquara - São Paulo - ...","R. Martinho Lutero, Jardim Campo Limpo - São P...",https://drivers.uber.com/earnings/trips/f1ceac...
0,UberX • 3 de out. de 2023 • 21:00,"R$ 18,86",19 min 23 segundos,12.1 km,"Avenida Casa Grande, Casa Grande - Diadema - S...","R. Geraldo Salini Romeo, Jardim Oriental - São...",https://drivers.uber.com/earnings/trips/301fb4...
0,UberX • 3 de out. de 2023 • 20:06,"R$ 69,09",43 min 34 segundos,32.7 km,"Rua Melvin Jones, Centro - Osasco - SP, 06010-...","Avenida Alberto Jafet, Vila Nogueira - Diadema...",https://drivers.uber.com/earnings/trips/b3c0f5...
0,UberX • 3 de out. de 2023 • 19:36,"R$ 24,03",20 min 1 segundos,13.5 km,"R. Werner Von Siemens, Lapa - São Paulo - SP, ...","R. José Gimenes Gomes, Vila Osasco - Osasco - ...",https://drivers.uber.com/earnings/trips/8305d8...
0,UberX • 3 de out. de 2023 • 17:02,"R$ 62,09",1 h 0 min,43.5 km,"Rua Padre Miguel de Campos, Artur Alvim - São ...","Rua Joaquim Guimarães, Raposo Tavares - São Pa...",https://drivers.uber.com/earnings/trips/482a13...


In [10]:

df.shape

(15784, 7)

In [12]:
df2 = pd.read_csv('dados/corridas_extraidas.csv', sep=';', encoding='utf-8')
df2.shape

(15784, 7)